# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/sleap_id_corrected/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")

In [18]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

In [54]:
LFP_SPECTRAL_DF = pd.read_pickle(os.path.join("./proc", FULL_LFP_TRACES_PKL))


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [55]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [56]:
LFP_SPECTRAL_DF.columns

Index(['tone_start_frame', 'reward_start', 'reward_dispensed',
       'tone_stop_frame', 'condition ', 'competition_closeness',
       'get_reward_frame', 'out_reward_frame', 'notes',
       'box_1_port_entry_frames',
       ...
       'trial_and_post_mPFC_gamma_phase', 'trial_and_post_mPFC_lfp_trace',
       'trial_and_post_mPFC_theta_band', 'trial_and_post_mPFC_theta_phase',
       'trial_and_post_vHPC_gamma_band', 'trial_and_post_vHPC_gamma_phase',
       'trial_and_post_vHPC_lfp_trace', 'trial_and_post_vHPC_theta_band',
       'trial_and_post_vHPC_theta_phase', 'trial_and_post_lfp_timestamps'],
      dtype='object', length=208)

In [57]:
all_freq_columns = [col for col in LFP_SPECTRAL_DF.columns if "all_frequencies_all_windows" in col]

In [58]:
for col in all_freq_columns:
    print(col)

trial_and_post_BLA_LH_coherence_all_frequencies_all_windows
trial_and_post_BLA_MD_coherence_all_frequencies_all_windows
trial_and_post_BLA_mPFC_coherence_all_frequencies_all_windows
trial_and_post_BLA_vHPC_coherence_all_frequencies_all_windows
trial_and_post_LH_MD_coherence_all_frequencies_all_windows
trial_and_post_LH_mPFC_coherence_all_frequencies_all_windows
trial_and_post_LH_vHPC_coherence_all_frequencies_all_windows
trial_and_post_MD_mPFC_coherence_all_frequencies_all_windows
trial_and_post_MD_vHPC_coherence_all_frequencies_all_windows
trial_and_post_mPFC_vHPC_coherence_all_frequencies_all_windows
trial_and_post_BLA_LH_granger_all_frequencies_all_windows
trial_and_post_BLA_MD_granger_all_frequencies_all_windows
trial_and_post_BLA_mPFC_granger_all_frequencies_all_windows
trial_and_post_BLA_vHPC_granger_all_frequencies_all_windows
trial_and_post_LH_BLA_granger_all_frequencies_all_windows
trial_and_post_LH_MD_granger_all_frequencies_all_windows
trial_and_post_LH_mPFC_granger_all_freq

In [59]:
LFP_SPECTRAL_DF[col].iloc[0].shape

(40, 56)

In [60]:
LFP_SPECTRAL_DF["competition_closeness"].unique()

array(['After trial', 'Subj 2 Only', 'Subj 2 blocking Subj 1',
       'Subj 2 then Subj 1', 'Close Call', 'Subj 1 then Subj 2',
       'Subj 1 Only', 'Subj 1 blocking Subj 2'], dtype=object)

In [61]:
OUTCOME_TO_COLOR = {"both_rewarded": "green", 
"novel_lose": "#e67073",
                    "novel_lose_competitive": "#e67073",
                    "novel_lose_no_comp": "#e67073",
"novel_win": "#93a5da",
                    "novel_win_competitive": "#93a5da",
                    "novel_win_no_comp": "#93a5da",
"lose": "#951a1d",
                    "lose_no_comp": "#951a1d",
                    "lose_competitive": "#951a1d",
"alone_rewarded": "#0499af",
"win": "#3853a3",
                    "win_competitive": "#3853a3",
                    "win_no_comp": "#3853a3",

"omission": "orange"}

In [62]:
session_types = ["training", "omission", "both", "novel"]

In [63]:
BAND_TO_FREQ_PLOT = {'theta': (4, 12), 'gamma': (30, 50)}
BAND_TO_FREQ_COLOR = {'theta': "red", 'beta': "blue", 'gamma': "green"}

In [64]:
preprocessing_type = ""

In [65]:
comp_closeness_dict = {'Subj 1 blocking Subj 2': "competitive",
'Subj 2 Only': "no_comp",
'Subj 2 blocking Subj 1': "competitive",
'Subj 1 then Subj 2': "competitive", 
'Subj 1 Only': "no_comp",
'Subj 2 then Subj 1': "competitive",
'Close Call': "competitive",
'After trial': "no_comp"}

In [68]:
LFP_SPECTRAL_DF["condition "] = LFP_SPECTRAL_DF["condition "].astype(str).apply(lambda x: x.strip())

In [69]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str).apply(lambda x: x.strip())

In [70]:
LFP_SPECTRAL_DF["agent"] = LFP_SPECTRAL_DF["agent"].astype(str).apply(lambda x: x.strip())

In [71]:
LFP_SPECTRAL_DF["trial_label"] = LFP_SPECTRAL_DF.apply(lambda x: "win" if x["condition "] == x["current_subject"] else ("lose" if x["condition "] == x["agent"] else "tie"), axis=1)

In [73]:
LFP_SPECTRAL_DF["competition_closeness"] = LFP_SPECTRAL_DF["competition_closeness"].map(comp_closeness_dict)

In [74]:
LFP_SPECTRAL_DF["competitive_trial_label"] = LFP_SPECTRAL_DF.apply(lambda x: "_".join([x["trial_label"], x["competition_closeness"]]), axis=1)


In [75]:
comp_to_style = {key: ("--" if "no_comp" in key else "-") for key in LFP_SPECTRAL_DF["competitive_trial_label"].unique() }

In [77]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF[LFP_SPECTRAL_DF["trial_label"] != "tie"]

In [76]:
comp_to_style

{'lose_no_comp': '--',
 'win_no_comp': '--',
 'lose_competitive': '-',
 'win_competitive': '-',
 'tie_competitive': '-'}

In [ ]:
LFP_SPECTRAL_DF["competition_closeness"]

In [72]:
LFP_SPECTRAL_DF["trial_label"]

0      lose
1       win
2      lose
3       win
4      lose
       ... 
243     win
244    lose
245     win
246     tie
247     tie
Name: trial_label, Length: 248, dtype: object

In [ ]:

for col in all_freq_columns:
    stripped_column = col.replace("trial_and_post_", "").replace("_all_frequencies_all_windows", "")
    for subj in LFP_SPECTRAL_DF["current_subject"].unique():
        current_subject_df = LFP_SPECTRAL_DF[LFP_SPECTRAL_DF["current_subject"] == subj]
        fig, ax = plt.subplots()
        ax.grid()
        plt.title("{} {}".format(stripped_column, subj))
        for index, row in current_subject_df.iterrows():
            plt.plot(np.nanmean(row[col], axis=0), color=OUTCOME_TO_COLOR[row["competitive_trial_label"]], linestyle=comp_to_style[row["competitive_trial_label"]])
        if "coherence" in col or "granger" in col:
            plt.ylim(0, 1)
        
        plt.savefig(os.path.join("./proc/trial_traces", "{}_{}.png".format(stripped_column, subj)))
    
        
        

'4.2'